In [1]:
import os
import numpy as np
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
# show all columns
pd.options.display.max_columns = None

## Join Bike volume data to links

In [3]:
# read in links csv
links = pd.read_csv(r".\Data\links.csv")

# read in links shapefile
links_shp = pd.DataFrame.spatial.from_featureclass(r".\Data\links.shp")
print(links.shape)
print(links_shp.shape)


(137281, 32)
(137281, 39)


In [4]:
# read in bike volume
bike_volume = pd.read_csv(r".\Data\bike_vol.csv")

#should be double the amount of links for both directions
print(bike_volume.shape)

# fill bike volume NAs with -1
bike_volume['bike_vol'] = bike_volume['bike_vol'].fillna(0)

(274562, 3)


In [5]:
# Create key to use for joining to links
bike_volume['key'] = np.where(bike_volume['from_node'].astype(int) < bike_volume['to_node'].astype(int), 
                              bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str), 
                              bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str))

# Create directional keys
bike_volume['ft_key'] = bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str)
bike_volume['tf_key'] = bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str)

bike_volume.head(15)

,from_node,to_node,bike_vol,key,ft_key,tf_key
0,0,732,0.741222,0_732,0_732,732_0
1,0,2638,0.455182,0_2638,0_2638,2638_0
2,1,190,0.528615,1_190,1_190,190_1
3,2,78,1.297461,2_78,2_78,78_2
4,3,4,0.204097,3_4,3_4,4_3
5,3,79,0.462335,3_79,3_79,79_3
6,3,859,1.203941,3_859,3_859,859_3
7,4,3,0.372698,3_4,4_3,3_4
8,4,5,0.433497,4_5,4_5,5_4
9,4,16,0.278498,4_16,4_16,16_4


In [6]:
# summarize trips in each direction
ft_vol_sum = pd.DataFrame(bike_volume.groupby('ft_key')['bike_vol'].sum())
tf_vol_sum = pd.DataFrame(bike_volume.groupby('tf_key')['bike_vol'].sum())

ft_vol_sum.columns = ['ft_bvol']
tf_vol_sum.columns = ['tf_bvol']

tf_vol_sum.head()

,tf_bvol
tf_key,
0_2638,0.455182
0_732,1.257331
100000_100021,0.000000
100000_100364,0.000000
100000_99978,0.000000


In [7]:
# summarize trips in both directions
volume_sum = pd.DataFrame(bike_volume.groupby('key')['bike_vol'].sum())
volume_sum.columns = ['total_bvol']
volume_sum.head(10)

,total_bvol
key,
0_2638,0.910364
0_732,1.998553
100000_100021,0.000000
100000_100364,0.000000
100001_100002,0.000000
100002_100026,0.000000
100003_100004,0.000000
100004_100019,0.000000
100004_100036,0.000000


In [8]:
#Create FTkey and TF key to use for joining to bike volumes
links['key'] = np.where(links['from_node'].astype(int) < links['to_node'].astype(int), 
                              links['from_node'].astype(str) + "_"+ links['to_node'].astype(str), 
                              links['to_node'].astype(str) + "_"+ links['from_node'].astype(str))

links[['from_node', 'to_node','key']].head(10)

,from_node,to_node,key
0,3,859,3_859
1,4,858,4_858
2,4,3,3_4
3,5,4,4_5
4,6,5,5_6
5,7,6,6_7
6,8,7,7_8
7,9,5,5_9
8,10,9,9_10
9,10,6,6_10


In [9]:
# copy the links table
links2 = links[['link_id', 'key']].copy()

# join the links with the bike volumes using the common keys
link_bike_vol = links2.merge(volume_sum, left_on='key', right_on='key', how='left')
link_bike_vol2 = link_bike_vol.merge(ft_vol_sum, left_on='key', right_on='ft_key', how='left')
link_bike_vol3 = link_bike_vol2.merge(tf_vol_sum, left_on='key', right_on='tf_key', how='left')

# examine the results
print(links2.shape)
print(link_bike_vol3.shape)

(137281, 2)
(137281, 5)


In [10]:
link_bike_vol3.head(5)

,link_id,key,total_bvol,ft_bvol,tf_bvol
0,0,3_859,2.407882,1.203941,1.203941
1,1,4_858,2.944378,1.472189,1.472189
2,2,3_4,0.576795,0.204097,0.372698
3,3,4_5,1.788281,0.433497,1.354784
4,4,5_6,1.810165,0.416916,1.393249


In [11]:
# Examine the column names
links_shp.columns

Index(['FID', 'Join_Count', 'TARGET_FID', 'FID_lines_', 'Name', 'Oneway',
       'Speed', 'AutoNetwor', 'BikeNetwor', 'PedNetwork', 'SourceData',
       'DriveTime', 'BikeTime', 'Pedestrian', 'Length_Mil', 'ConnectorN',
       'RoadClass', 'AADT', 'AADT_YR', 'BIKE_L', 'BIKE_R', 'Shape_Leng', 'id',
       'Start_Key', 'End_Key', 'Bike_Lane', 'Bike_Path', 'Bike_Blvd', 'SIGID',
       'Signal', 'BUFF_DIST', 'ORIG_FID', 'link_id', 'from_z', 'to_z',
       'Slope_AB', 'Slope_BA', 'Slope_Per', 'SHAPE'],
      dtype='object')

In [12]:
# export final result to csv
link_bike_vol3['link_id'] = link_bike_vol3['link_id'].astype('int64')

# join bike vol to links shapefile
links4 = links_shp.merge(link_bike_vol3, left_on='link_id', right_on='link_id', how='outer')

# export to shape
links4.spatial.to_featureclass(location=r".\Outputs\links_bv.shp")


'E:\\Projects\\utah_bike_demand_model\\Post_Process_Bike_Model_Outputs\\Outputs\\links_bv.shp'

## Summarize zone trips by Attracting/Producing Zone

In [13]:
# read in zones
zones = pd.DataFrame.spatial.from_featureclass(r".\Data\microzones.shp")
zones.head()

,FID,Id,zone_id,co_tazid,tazid,co_fips,co_name,residentia,households,population,jobs1,jobs3,jobs4,jobs5,jobs6,jobs7,jobs9,jobs10,avgincome,enrol_elem,enrol_midl,enrol_high,hhsize_lc1,hhsize_lc2,hhsize_lc3,pct_poplc1,pct_poplc2,pct_poplc3,pct_ag1,pct_ag2,pct_ag3,inc1,inc2,inc3,inc4,park_score,park_area,school_cd,coll_enrol,th_score,comm_rail,light_rail,gqu_ratio,node_id,jobs_total,mixed_use,AREA_SQMIL,SHAPE
0,0,0,0,491871,1871,49,UTAH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52669.0,0.0,0.0,0.0,1.0725,2.4375,1.0920,0.1170,0.8570,0.026,0.2925,0.6880,0.0195,0.239226,0.240608,0.300300,0.219866,0,0,0,0,0,0,0,0.0,17478,0.0,0.0,0.061776,"{""rings"": [[[416159.7923999997, 4462258.2762],..."
1,1,0,1,350671,671,35,SALT LAKE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33251.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,1.0000,0.000,0.0000,1.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,54521,0.0,0.0,0.054373,"{""rings"": [[[401359.7923999997, 4511858.2762],..."
2,2,0,2,492677,2677,49,UTAH,0.0,5.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65957.0,0.0,0.0,0.0,1.7692,4.0053,1.8792,0.2007,0.6783,0.121,0.3038,0.5852,0.1110,0.153432,0.209589,0.325339,0.311640,0,0,0,0,0,0,0,0.0,3070,0.0,0.0,0.061776,"{""rings"": [[[436159.7923999997, 4437858.2762],..."
3,3,0,3,492158,2158,49,UTAH,0.0,85.0,376.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76749.0,0.0,0.0,0.0,2.4000,5.3100,2.4800,0.1800,0.7600,0.060,0.4200,0.5300,0.0500,0.106071,0.182161,0.325879,0.385888,0,0,0,0,0,0,0,0.0,25213,0.0,0.0,0.061776,"{""rings"": [[[435759.7923999997, 4474258.2762],..."
4,4,0,4,351384,1384,35,SALT LAKE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88520.0,0.0,0.0,0.0,1.8900,4.5200,1.8750,0.2600,0.6800,0.060,0.4200,0.5400,0.0400,0.072931,0.150915,0.310025,0.466129,0,0,0,0,0,0,0,0.0,28595,0.0,0.0,0.061776,"{""rings"": [[[412559.7923999997, 4489058.2762],..."


### Read in trip tables, summarize, and format

In [14]:
def summarize_zones(trips_df, name):
    
    # summarize trips by attraction or production
    trips_sum_attr = pd.DataFrame(trips_df.groupby('azone')['bk'].sum())
    trips_sum_prod = pd.DataFrame(trips_df.groupby('pzone')['bk'].sum())
    
    # format tables
    trips_sum_attr['zone_id'] = trips_sum_attr.index
    trips_sum_attr.columns = [name + '_abk', 'zone_id']
    trips_sum_prod['zone_id'] = trips_sum_prod.index
    trips_sum_prod.columns = [name + '_pbk', 'zone_id']
    
    # join the attraction and production summary tables using zone id
    merged = trips_sum_attr.merge(trips_sum_prod, left_on='zone_id', right_on='zone_id', how='outer')
    return merged
    

In [15]:
# sch_univ = pd.read_csv(r".\Data\sch_univ_trip.csv")
# sch_univ_sum = summarize_zones(sch_univ, 'univ')
# sch_univ_sum.isnull().values.any()

In [16]:
# Discretionary trips (social trips, some recreation)
disc = pd.read_csv(r".\Data\disc_trip.csv")
disc_sum = summarize_zones(disc, 'disc')
del disc

# Maintenance trips (e.g. groceries)
maint = pd.read_csv(r".\Data\maint_trip.csv")
maint_sum = summarize_zones(maint, 'mnt')
del maint

# Maintenance trips non-home-based (e.g. groceries)
maint_nhb = pd.read_csv(r".\Data\maint_trip_nhb.csv")
maint_nhb_sum = summarize_zones(maint_nhb, 'mntnhb')
del maint_nhb

# Recreational family trips
rec_fam = pd.read_csv(r".\Data\rec_fam_trip.csv")
rec_fam_sum = summarize_zones(rec_fam, 'recfam')
del rec_fam

# Recreation long trips
rec_long = pd.read_csv(r".\Data\rec_long_trip.csv")
rec_long_sum = summarize_zones(rec_long, 'reclng')
del rec_long

# Recreation other trips (recreation that doesn't fall into family or long)
rec_oth = pd.read_csv(r".\Data\rec_oth_trip.csv")
rec_oth_sum = summarize_zones(rec_oth, 'recoth')
del rec_oth

# school (grade) trips
sch_grade = pd.read_csv(r".\Data\sch_grade_trip.csv")
sch_grade_sum = summarize_zones(sch_grade, 'grade')
del sch_grade

# school (university) trips
sch_univ = pd.read_csv(r".\Data\sch_univ_trip.csv")
sch_univ_sum = summarize_zones(sch_univ, 'univ')
del sch_univ

# Work trips
work = pd.read_csv(r".\Data\work_trip.csv")
work_sum = summarize_zones(work, 'wrk')
del work

# Work non-home-based trips
work_nhb = pd.read_csv(r".\Data\work_trip_nhb.csv")
work_nhb_sum = summarize_zones(work_nhb, 'wrknhb')
del work_nhb

In [17]:
rec_fam_sum

,recfam_abk,zone_id,recfam_pbk
0,0.000559,0,NaN
1,0.001262,1,NaN
2,0.029785,2,0.046346
3,0.207457,3,0.808998
4,0.003285,4,NaN
...,...,...,...
14530,1.389145,15052,0.129528
14531,1.091482,15053,0.614856
14532,1.852708,15054,0.386745
14533,0.633416,15055,1.679401


### Merge trip summarizes back to microzone shapefile

In [18]:
# Create a clean copy of zones dataset
zones2 = zones[['zone_id', 'co_tazid', 'tazid', 'co_fips', 'co_name', 'SHAPE']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')



# Join trip tables
zones2 = zones2.merge(disc_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(maint_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(maint_nhb_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_fam_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_long_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_oth_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(sch_grade_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(sch_univ_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(work_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(work_nhb_sum, left_on='zone_id', right_on='zone_id', how='left')




# preview table
zones2.head(30)

,zone_id,co_tazid,tazid,co_fips,co_name,SHAPE,disc_abk,disc_pbk,mnt_abk,mnt_pbk,mntnhb_abk,mntnhb_pbk,recfam_abk,recfam_pbk,reclng_abk,reclng_pbk,recoth_abk,recoth_pbk,grade_abk,grade_pbk,univ_abk,univ_pbk,wrk_abk,wrk_pbk,wrknhb_abk,wrknhb_pbk
0,0,491871,1871,49,UTAH,"{'rings': [[[416159.7923999997, 4462258.2762],...",0.016953,NaN,0.054621,NaN,0.022831,NaN,0.000559,NaN,0.000006,NaN,0.010495,NaN,0.040384,NaN,1.891286e-05,NaN,0.033553,NaN,0.025601,NaN
1,1,350671,671,35,SALT LAKE,"{'rings': [[[401359.7923999997, 4511858.2762],...",0.009816,NaN,0.029895,NaN,0.012496,NaN,0.001262,NaN,0.000012,NaN,0.007415,NaN,0.039251,NaN,7.821718e-03,NaN,0.007342,NaN,0.005602,NaN
2,2,492677,2677,49,UTAH,"{'rings': [[[436159.7923999997, 4437858.2762],...",0.128620,0.197668,0.292830,0.323520,0.122403,0.135231,0.029785,0.046346,0.000065,0.126136,0.058318,NaN,0.205540,0.160164,7.104136e-03,NaN,0.075069,0.341359,0.057278,0.260457
3,3,492158,2158,49,UTAH,"{'rings': [[[435759.7923999997, 4474258.2762],...",0.710326,2.225234,0.988533,4.692832,0.413207,1.961604,0.207457,0.808998,0.000028,2.369456,0.048804,NaN,0.144761,2.997537,1.261751e-02,NaN,0.039325,4.963077,0.030005,3.786828
4,4,351384,1384,35,SALT LAKE,"{'rings': [[[412559.7923999997, 4489058.2762],...",0.045710,NaN,0.127270,NaN,0.053199,NaN,0.003285,NaN,0.000023,NaN,0.041844,NaN,0.212584,NaN,1.403655e-02,NaN,0.042024,NaN,0.032064,NaN
5,5,492483,2483,49,UTAH,"{'rings': [[[447359.7923999997, 4449458.2762],...",0.177746,NaN,0.448836,NaN,0.187613,NaN,0.015515,NaN,0.000154,NaN,0.138948,NaN,0.564915,NaN,2.440457e-01,NaN,0.133122,NaN,0.101572,NaN
6,6,491856,1856,49,UTAH,"{'rings': [[[408559.7923999997, 4468258.2762],...",0.064930,NaN,0.277848,NaN,0.116141,NaN,0.003338,NaN,0.000041,NaN,0.150096,NaN,0.171750,NaN,1.148087e-07,NaN,0.219767,NaN,0.167682,NaN
7,7,350795,795,35,SALT LAKE,"{'rings': [[[414959.7923999997, 4509458.2762],...",7.454291,NaN,6.506838,NaN,2.719858,NaN,0.008482,NaN,0.000074,NaN,7.267891,NaN,8.089714,NaN,7.412790e-01,NaN,23.029715,NaN,17.571672,NaN
8,8,492141,2141,49,UTAH,"{'rings': [[[432959.7923999997, 4475058.2762],...",0.332538,2.703024,0.494449,4.645803,0.206679,1.941946,0.778772,0.665109,1.337381,2.023570,0.224811,6.618056,0.101535,2.637736,1.702777e-03,NaN,0.024255,14.797826,0.018507,11.290741
9,9,492179,2179,49,UTAH,"{'rings': [[[434159.7923999997, 4469458.2762],...",5.626985,6.599478,8.578693,9.873044,3.585894,4.126933,0.294984,1.361801,0.000024,3.530660,1.524816,NaN,2.156188,3.487781,6.999176e-03,0.663494,14.754801,7.302547,11.257913,5.571843


In [19]:
zones2.columns

Index(['zone_id', 'co_tazid', 'tazid', 'co_fips', 'co_name', 'SHAPE',
       'disc_abk', 'disc_pbk', 'mnt_abk', 'mnt_pbk', 'mntnhb_abk',
       'mntnhb_pbk', 'recfam_abk', 'recfam_pbk', 'reclng_abk', 'reclng_pbk',
       'recoth_abk', 'recoth_pbk', 'grade_abk', 'grade_pbk', 'univ_abk',
       'univ_pbk', 'wrk_abk', 'wrk_pbk', 'wrknhb_abk', 'wrknhb_pbk'],
      dtype='object')

In [20]:
# fill na's with 0
zones3 = zones2

# calc totals        
zones3['total_abk'] = (zones3['disc_abk'] + zones3['mnt_abk'] + zones3['mntnhb_abk'] + 
                       zones3['recfam_abk'] + zones3['reclng_abk'] + zones3['recoth_abk'] + 
                       zones3['grade_abk'] + zones3['univ_abk'] + zones3['wrk_abk'] + zones3['wrknhb_abk']) 

zones3['total_pbk'] = (zones3['disc_pbk'] + zones3['mnt_pbk'] + zones3['mntnhb_pbk'] + 
                       zones3['recfam_pbk'] + zones3['reclng_pbk'] + zones3['recoth_pbk'] + 
                       zones3['grade_pbk'] + zones3['univ_pbk'] + zones3['wrk_pbk'] + zones3['wrknhb_pbk']) 

# fill NAs where necessary
for field in list(zones3.columns):
    if field not in ['SHAPE']:
        zones3[field].fillna(0, inplace=True)

In [21]:
# then export to shape
zones3.spatial.to_featureclass(location=r".\Outputs\Microzone_Trip_Summaries.shp")

'E:\\Projects\\utah_bike_demand_model\\Post_Process_Bike_Model_Outputs\\Outputs\\Microzone_Trip_Summaries.shp'

## Merge zone attraction and production scores with the microzone geometry

In [22]:
# Create a clean copy of zones dataset
zones2 = zones[['zone_id', 'co_tazid', 'tazid', 'co_fips', 'co_name', 'SHAPE']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')

# need to add zone_id to empty field in output csv
ascore = pd.read_csv(r".\Data\zone_attraction_size.csv")
pscore = pd.read_csv(r".\Data\zone_production_size.csv")

zones3a = zones2.merge(ascore, left_on='zone_id', right_on='zone_id', how='left')
zones3p = zones2.merge(pscore, left_on='zone_id', right_on='zone_id', how='left')

In [23]:
# fill NAs where necessary
for field in list(zones3a.columns):
    if field !='SHAPE':
        zones3a[field].fillna(-1, inplace=True)

# fill NAs where necessary
for field in list(zones3p.columns):
    if field !='SHAPE':
        zones3p[field].fillna(-1, inplace=True)

# Fill NAs with -1, then export to shape
zones3a.spatial.to_featureclass(location=r".\Outputs\Microzone_A_Scores.shp")
zones3p.spatial.to_featureclass(location=r".\Outputs\Microzone_P_Scores.shp")

'E:\\Projects\\utah_bike_demand_model\\Post_Process_Bike_Model_Outputs\\Outputs\\Microzone_P_Scores.shp'

## Get Centroid Nodes

In [24]:
nodes = pd.DataFrame.spatial.from_featureclass(r".\Data\nodes.shp")
nodes['node_id'] = nodes.index
nodes.shape

(108655, 29)

In [25]:
nodes2 = nodes[['node_id', 'xcoord', 'ycoord', 'zcoord', 'SHAPE']].copy()
centroids = nodes2.merge(zones[['node_id', 'zone_id']], left_on='node_id', right_on='node_id', how='inner')
print(centroids.columns)

Index(['node_id', 'xcoord', 'ycoord', 'zcoord', 'SHAPE', 'zone_id'], dtype='object')


In [26]:
centroids = centroids[['node_id', 'xcoord', 'ycoord', 'zcoord', 'zone_id', 'SHAPE']].copy()
centroids.spatial.to_featureclass(location=r".\Outputs\Microzone_Centroids.shp")

'E:\\Projects\\utah_bike_demand_model\\Post_Process_Bike_Model_Outputs\\Outputs\\Microzone_Centroids.shp'